## Define Imports

In [16]:
import math # Mathematical functions 
import numpy as np # Fundamental package for scientific computing with Python
import pandas as pd # Additional functions for analysing and manipulating data
from datetime import date, timedelta, datetime # Date Functions
from pandas.plotting import register_matplotlib_converters # This function adds plotting functions for calender dates
import matplotlib.pyplot as plt # Important package for visualization - we use this to plot the market data
import matplotlib.dates as mdates # Formatting dates
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error # Packages for measuring model performance / errors
from tensorflow.keras import Sequential # Deep learning library, used for neural networks
from tensorflow.keras.layers import LSTM, Dense, Dropout # Deep learning classes for recurrent and regular densely-connected layers
from tensorflow.keras.callbacks import EarlyStopping # EarlyStopping during model training
from sklearn.preprocessing import RobustScaler, MinMaxScaler # This Scaler removes the median and scales the data according to the quantile range to normalize the price data 
import seaborn as sns # Visualization
sns.set_style('white', { 'axes.spines.right': False, 'axes.spines.top': False})
%matplotlib inline
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from sklearn.preprocessing import MinMaxScaler

In [15]:
!pip install tensorflow

     -------------------------------------- 455.9/455.9 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 14.2/14.2 MB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 895.9/895.9 kB 6.3 MB/s eta 0:00:00
     -------------------------------------- 124.6/124.6 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 438.7/438.7 kB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 6.0 MB/s eta 0:00:00
     -------------------------------------- 781.3/781.3 kB 7.1 MB/s eta 0:00:00
     ---------------------------------------- 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.10.1 requires protobuf<5.0.0dev,>=3.20.1, but you have protobuf 3.19.6 which is incompatible.


In [12]:
import yfinance
msft = yf.download(MSFT, start=start_date, end=end_date)
hist = msft.history(period='max')
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.055380,0.063524,0.055380,0.060809,1031788800,0.00,0.0
1986-03-14 00:00:00-05:00,0.060809,0.064067,0.060809,0.062980,308160000,0.00,0.0
1986-03-17 00:00:00-05:00,0.062980,0.064610,0.062980,0.064067,133171200,0.00,0.0
1986-03-18 00:00:00-05:00,0.064067,0.064610,0.061894,0.062437,67766400,0.00,0.0
1986-03-19 00:00:00-05:00,0.062437,0.062980,0.060809,0.061351,47894400,0.00,0.0
...,...,...,...,...,...,...,...
2022-11-11 00:00:00-05:00,242.307141,247.293090,241.250107,246.415558,34600900,0.00,0.0
2022-11-14 00:00:00-05:00,241.309951,243.224554,238.537765,240.871185,31123300,0.00,0.0
2022-11-15 00:00:00-05:00,244.969641,246.305872,239.355458,241.290009,31390100,0.00,0.0


In [6]:
MSFT = pd.read_csv("MSFT.csv")
# MSFT = MSFT.iloc[7000:,:]
MSFT;
display(MSFT)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1986-03-13,0.088542,0.101562,0.088542,0.097222,0.062378,1031788800
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.064606,308160000
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.065720,133171200
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.064049,67766400
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.062935,47894400
...,...,...,...,...,...,...,...
8579,2020-03-26,148.399994,156.660004,148.369995,156.110001,156.110001,64568100
8580,2020-03-27,151.750000,154.889999,149.199997,149.699997,149.699997,57042300
8581,2020-03-30,152.440002,160.600006,150.009995,160.229996,160.229996,63420300
8582,2020-03-31,159.399994,164.779999,156.559998,157.710007,157.710007,77927200


In [ ]:
# Plot line charts
df_plot = MSFT.copy()

ncols = 3
nrows = int(round(df_plot.shape[1] / ncols, 0))

fig, ax = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, figsize=(14, 7))
for i, ax in enumerate(fig.axes):
        sns.lineplot(data = df_plot.iloc[:, i], ax=ax)
        ax.tick_params(axis="x", rotation=30, labelsize=10, length=0)
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
fig.tight_layout()
plt.show()

In [7]:
class Basenet():
    def __init__(self):
        self.train_set = None
        self.val_set = None
        self.test_set = None
    def split_data(self,stock, lookback,splitsize):
        data_raw = stock.to_numpy() # convert to numpy array
        data = []
        # create all possible sequences of length seq_len
        for index in range(len(data_raw) - lookback): 
            data.append(data_raw[index: index + lookback])
        #calc test size
        data = np.array(data)
        test_size = int(np.round(len(data)*splitsize))
        bg_train_size = len(data) - test_size
        val_size = int(np.round(bg_train_size*(splitsize/2)))
        train_size = bg_train_size - val_size
        # test_set_size = int(np.round(0.2*data.shape[0]))
        # train_set_size = data.shape[0] - (test_set_size)
        print(f"Train :{train_size}, Val :{val_size}, test :{test_size}")
        print("bg_train :",bg_train_size)
        self.train_set = data[:train_size,:]
        self.val_set = data[train_size:bg_train_size,:]
        self.test_set = data[bg_train_size:,:]
#         x_train = data[:train_size,:-1]
#         y_train = data[:train_size,-1]
        print("Training Size :",len(self.train_set))
        print("Validation Size :",len(self.val_set))
#         x_test = data[train_size:,:-1]
#         y_test = data[train_size:,-1]
        print("Testing Size :",len(self.test_set))
        # return [x_train, y_train, x_test, y_test]
        return True


  

In [8]:
l = Basenet()
l.split_data(MSFT, 20,.2)

Train :6166, Val :685, test :1713
bg_train : 6851
Training Size : 6166
Validation Size : 685
Testing Size : 1713


True